In [4]:
!pip install snowflake-connector-python
!pip install snowflake-snowpark-python
!pip install pandas
!pip install pyarrow
!pip install matplotlib
!pip install seaborn
!pip install numpy
!pip install scipy
!pip install scikit-learn
!pip install statsmodels
!pip install tensorflow
!pip install keras
!pip install xgboost
!pip install lightgbm
!pip install catboost
!pip install mlflow
!pip install pydot

You should consider upgrading via the '/Users/navisha/Desktop/PROJECTS/machine-learning-with-snowpark/.venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/navisha/Desktop/PROJECTS/machine-learning-with-snowpark/.venv/bin/python3 -m pip install --upgrade pip' command.
  Using cached pandas-2.2.3-cp39-cp39-macosx_11_0_arm64.whl (11.3 MB)
     |████████████████████████████████| 347 kB 10.0 MB/s eta 0:00:01
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl (5.3 MB)
You should consider upgrading via the '/Users/navisha/Desktop/PROJECTS/machine-learning-with-snowpark/.venv/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 30.7 MB 6.2 MB/s eta 0:00:01
You should consider upgrading via the '/Users/navisha/Desktop/PROJECTS/machine-learning-with-snowpark/.venv/bin/python3 -m pip install --upgrade pip' command.
     |████████████████████████████████| 7.8 MB 7.7 MB/s eta 0:00:01
     |██████████████

### IMPORTS

In [2]:
from snowflake.snowpark.session import Session
import getpass
import pandas as pd
import json
import os

/Users/navisha/Desktop/PROJECTS/machine-learning-with-snowpark/.venv/lib/python3.9/site-packages/snowflake/connector/options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.1), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


### CREATE SNOWPARK SESSION

In [3]:
# WGDNNWJ-OD50189
accountname = getpass.getpass() # ORGNAME-ACCOUNTNAME (separated by minus sign)
print(accountname)

WGDNNWJ-OD50189


In [4]:
username = getpass.getpass()    # SNOWFLAKE-USERNAME
password = getpass.getpass()    # SNOWFLAKE-PASSWORD

In [5]:
connection_parameters = {
    "account": accountname,
    "user": username,
    "password": password,
    "role": "ACCOUNTADMIN"
}

session = Session.builder.configs(connection_parameters).create()

### Create Virtual Warehouse (Compute) 

In [6]:
# Create two Virtual Warehouses
session.sql("CREATE OR REPLACE WAREHOUSE CC_WH_HIGH_MEM WITH WAREHOUSE_SIZE='MEDIUM' WAREHOUSE_TYPE='SNOWPARK-OPTIMIZED'").collect()
session.sql("CREATE OR REPLACE WAREHOUSE CC_WH WITH WAREHOUSE_SIZE='X-SMALL'").collect()

[Row(status='Warehouse CC_WH successfully created.')]

### Create Database in Snowflake

In [7]:
# Create Databse
session.sql("CREATE OR REPLACE DATABASE CC_DB").collect()

[Row(status='Database CC_DB successfully created.')]

### Load Kaggle Dataset into Snowflake DB

In [8]:
# Loading from local CSV-files
application_record_df = pd.read_csv('RawData/application_record.csv.zip')
credit_record_df = pd.read_csv('RawData/credit_record.csv.zip')

In [9]:
# Upload to Snowflake
session.write_pandas(application_record_df, table_name='APPLICATION_RECORD', auto_create_table=True, overwrite=True)
session.write_pandas(credit_record_df, table_name='CREDIT_RECORD', auto_create_table=True, overwrite=True)

In [10]:
connection_parameters["database"] = 'CC_DB'
connection_parameters["schema"] = 'PUBLIC'
connection_parameters["warehouse"] = 'CC_WH'
with open('creds.json', 'w') as outfile:
        outfile.write(json.dumps(connection_parameters))

In [11]:
session.close()